In [2]:
import gpflow
import numpy as np
import math
import tensorflow as tf
import get_history as gh

In [34]:
sess = tf.Session()

In [57]:
 def get_all_sigmas(sigma_v,g_one,g_two,day_one,day_two):
    #sigmax = [[] for i in range(100)]
    #tf.placeholder(tf.float64, shape=(100, 2,2))
    
            
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        #effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
    
    
    
        
        
        get_g = g_one
        get_g_two = g_two
        
        effects_onef = [[] for i in range(100)]
        effects_twof = [[] for i in range(100)]

        
        to_return = [[] for i in range(100)]
        for i in range(100):
            temp = [[] for i in range(100)]
            tempeo = [[] for i in range(100)]
            tempet = [[] for i in range(100)]
            for j in range( 100):
                rho = rbf_custom(day_one[i], X2=day_two[j])
            #print(sess.run(rho))
                val = rho[0]
        #tf.scalar_mul(val,sigma_v)
            #print(val)
                this_sigma = tf.scalar_mul(val,sigma_v)
            #print(this_sigma)
                effects_one = tf.tensordot(get_g[i],this_sigma,axes=[[0],[1]])
                effects_two = tf.tensordot(effects_one,get_g_two[j],axes = [[0],[0]])
                
                tempeo[j]=tf.tensordot(get_g[i],sigma_v,axes=[[0],[1]])
                tempet[j]=tf.tensordot(tf.tensordot(get_g[i],sigma_v,axes=[[0],[1]]),get_g_two[j],axes = [[0],[0]])
                
                temp[j]=effects_two
                
            to_return[i]=temp
            effects_onef[i]=tempeo
            effects_twof[i]=tempet
            
            
        return to_return,effects_onef,effects_twof 

In [4]:
##need to reformat history
def put_x_together(h):
    X = []
    
    for i in range(len(h.states)):
        state_vector = h.states[i]
        state_vector.append(float(h.users[i]))
        state_vector.append(float(h.day_function[i]))
        X.append(state_vector)
    return X

In [5]:
history = gh.get_history(5,100,5)

In [6]:
X = put_x_together(history)
X=np.array(X)

In [7]:
def gather_cols(params, indices, name=None):
    """Gather columns of a 2D tensor.

    Args:
        params: A 2D tensor.
        indices: A 1D tensor. Must be one of the following types: ``int32``, ``int64``.
        name: A name for the operation (optional).

    Returns:
        A 2D Tensor. Has the same type as ``params``.
    """
    with tf.op_scope([params, indices], name, "gather_cols") as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
        except ValueError:
            raise ValueError('\'params\' must be 1D.')

        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],
                                       [-1, 1]) + indices, [-1])
        return tf.reshape(tf.gather(p_flat, i_flat),
                          [p_shape[0], -1])

In [8]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
rdayone = [x[6] for x in X]
rdaytwo = rdayone
rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(100)] for i in range(100)])


In [20]:
    def rbf_custom( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return tf.exp(-tf.divide(tf.square(tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        #return tf.constant(1.0,dtype=tf.float64)

In [29]:
rbf_custom_np( 1, X2=5)

1.1253517471925912e-07

In [35]:
rbf_custom(tf.constant([1.0],dtype=tf.float64),tf.constant([5.0],dtype=tf.float64)).eval(session=sess)

array([1.12535175e-07])

In [9]:
Xt = tf.constant(X)
g_one = gather_cols(Xt, [3,4], name=None)

In [10]:
g_two=g_one

In [11]:
sigma_v =  tf.constant(np.array([[10.0,0.0],[0.0,10.0]]))

In [15]:
effects_one = tf.tensordot(tf.transpose(g_one),sigma_v,axes=[[0],[1]])
effects_two = tf.tensordot(effects_one,tf.transpose(g_two),axes = [[1],[0]])
effects_three = tf.matmul(effects_two,rhos)

In [59]:
effects_four = tf.multiply(effects_two,rhos)

In [23]:
day_one = tf.constant(rdayone,dtype=tf.float64)
day_two = tf.constant(rdaytwo,dtype=tf.float64)

In [27]:
day_two = gather_cols(Xt, [6], name=None)
day_one = gather_cols(Xt, [6], name=None)


In [58]:
test = get_all_sigmas(sigma_v,g_one,g_two,day_one,day_two)

In [41]:
ef = sess.run(test[0])

In [46]:
np.array(ef)[0]

array([2.61161526e+000, 9.11904516e-001, 2.62467820e+000, 2.44816333e+000,
       9.79957842e-001, 2.59757154e+000, 2.56848822e+000, 4.66262582e-002,
       9.99542392e-001, 4.77759216e-002, 3.20653571e-004, 9.36039750e-001,
       4.88592941e-002, 4.71598828e-002, 3.07013339e-004, 3.38637508e-004,
       2.62522934e-007, 3.45447606e-011, 6.16160387e-016, 9.50889585e-001,
       3.11277089e-007, 3.32562799e-004, 4.54491476e-002, 2.71452458e-007,
       1.31902385e-021, 4.12423089e-028, 3.20429153e-004, 3.80123276e-011,
       1.77092716e-035, 6.08681502e-016, 1.31709134e-021, 4.19586835e-028,
       2.99624932e-007, 9.64268681e-044, 3.28498901e-011, 3.47110259e-011,
       5.94896177e-016, 6.79513350e-053, 1.28596779e-021, 6.13181664e-016,
       4.29147968e-028, 2.80012813e-007, 1.40721971e-021, 7.17450248e-063,
       4.03915698e-028, 1.77700722e-035, 9.97442177e-044, 7.22995372e-053,
       1.71482030e-035, 1.00287915e-073, 7.00314720e-063, 9.22323026e-044,
       7.83708175e-053, 1

In [47]:
cef = sess.run(effects_three)

In [50]:
eone = sess.run(test[1])

In [53]:
ceone = sess.run(effects_one)

In [56]:
rhos[0]

array([0.38520803, 0.33582183])

In [55]:
np.array(ceone)[0]

array([3.85208027, 3.35821831])

In [52]:
np.array(eone)[0]

array([[3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.85208027, 3.35821831],
       [3.

In [60]:
eff = sess.run(effects_four)

In [61]:
np.array(eff)[0]

array([2.61161526e+000, 9.11904516e-001, 2.62467820e+000, 2.44816333e+000,
       9.79957842e-001, 2.59757154e+000, 2.56848822e+000, 4.66262582e-002,
       9.99542392e-001, 4.77759216e-002, 3.20653571e-004, 9.36039750e-001,
       4.88592941e-002, 4.71598828e-002, 3.07013339e-004, 3.38637508e-004,
       2.62522934e-007, 3.45447606e-011, 6.16160387e-016, 9.50889585e-001,
       3.11277089e-007, 3.32562799e-004, 4.54491476e-002, 2.71452458e-007,
       1.31902385e-021, 4.12423089e-028, 3.20429153e-004, 3.80123276e-011,
       1.77092716e-035, 6.08681502e-016, 1.31709134e-021, 4.19586835e-028,
       2.99624932e-007, 9.64268681e-044, 3.28498901e-011, 3.47110259e-011,
       5.94896177e-016, 6.79513350e-053, 1.28596779e-021, 6.13181664e-016,
       4.29147968e-028, 2.80012813e-007, 1.40721971e-021, 7.17450248e-063,
       4.03915698e-028, 1.77700722e-035, 9.97442177e-044, 7.22995372e-053,
       1.71482030e-035, 1.00287915e-073, 7.00314720e-063, 9.22323026e-044,
       7.83708175e-053, 1